In [ ]:
# TODO: NER
# TODO: Word Sense Disambiguation
# TODO: NEL
# TODO: Cannonicalisation
# TODO: Open IE
# TODO: Opinion Triples
# TODO: Opinion Analysis, Visualisation, Maps: Entities, Similarity Clustering
# TODO: Knowledge Graph
# TODO: Neo4j
# TODO: Operationalise persist Cloud DB + App interface

In [ ]:
import stanza
import pandas as pd
import json

In [ ]:
# Stanza Pipeline
#stanza.download("en")
nlp = stanza.Pipeline(lang="en", processors="tokenize, mwt, pos, ner")

In [ ]:
all_news = [json.loads(_) for _ in open("data/RSS_objects.jsonl")]

In [ ]:
entities = []

for news in all_news:
    doc = nlp(news["summary"])

    for sent in doc.sentences:
        entity_id = len(entities)

        for ent in sent.ents:
            for _ in entities:
                found = False

                if _['text'] == ent.text and _['type'] == ent.type:
                    found = True
                    _["news_ids"].append(news["id"])
                    _["count"] += 1

                if 'entities' in news:
                    news['entities'].append(_['id'])

                else:
                    news['entities'] = [_['id']]

            if not found:
                entity = {
                    "id": entity_id,
                    "text": ent.text,
                    "type": ent.type,
                    "count": 1,
                    "news_ids": [news["id"]]

                }

                entities.append(entity)

                if "entities" in news:
                    news["entities"].append(entity_id)

                else:
                     news["entities"] = [entity_id]

                entity_id = entity_id + 1

In [ ]:
entities

In [ ]:
df = pd.DataFrame(data=entities)
df

In [ ]:
import sparql

entity_types = ['PERSON','NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW']

s = sparql.Service("http://dbpedia.org/sparql", "GET")

In [ ]:
for entity in entities:

    if entity["type"] in entity_types:
        searchText = entity['text'].replace('"', '')
        curQuery = 'SELECT DISTINCT * WHERE '
        curQuery = curQuery + '{ ?url rdfs:label "' + searchText + '"@en . '
        curQuery = curQuery + 'FILTER(STRSTARTS(str(?url), "http://dbpedia.org/resource/"))}'

        try:
            results = s.query(curQuery).fetchall()

        except:
            continue

        if len(results) > 0:
            for result in results:
                resourceName = sparql.unpack_row(result)[0]
                if 'Category:' not in resourceName and 'property' not in resourceName:
                    print(resourceName)
                    entity['dbpedia_Resource'] = resourceName

In [ ]:
# Spotlight
import requests
from IPython.core.display import display, HTML

spotlight_url = "http://api.dbpedia-spotlight.org/en/annotate"

headers = {
    "accept": "application/json"
}

for _ in all_news:
    params = {
        "text": _["title"],
        "confidence": 0.35
    }
    try:
        res = requests.get(spotlight_url, params=params, headers=headers)
        _["annotation"] = res.json()

    except:
        _["annotation"] = "NIL"
        continue

In [ ]:
all_news[1]

In [ ]:
# TODO: Mask
# TODO: Advanced Meta Data

item = all_news[5]

def locate(text, surface_form):
    items = text.split()
    len_ = len(surface_form.split())

    for idx, _ in enumerate(items):
        if len_ > 1:
            surface_form = surface_form.split()[0]

        if _.startswith(surface_form):
            return (idx, idx + len_)

        else: None

count = 0
for item in all_news:
    count += 1

    #print(item["id"], item["title"])

    if item["annotation"] == "NIL":
        continue

    else:

        link_text = item["title"].split()

        if "Resources" in item["annotation"]:
            for ent in item["annotation"]["Resources"]:
                surface_form = ent["@surfaceForm"]
                uri = ent["@URI"]

                try:
                    start, finish = locate(item["title"], surface_form)

                except:
                    continue

                if start != finish:
                    # Slice Assignment:
                    link_text[start: finish] = [f"[{surface_form}, {uri}]"]

                elif start == finish:
                    link_text[start] = f"[{surface_form}, {uri}]"

            disambiguated = " ".join(_ for _ in link_text)
            print(item["title"])
            print(disambiguated, "\n")

        else: continue